<a href="https://colab.research.google.com/github/kessingtonosazee/GCP_Project_1/blob/master/Linear_Regression2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Predicting Price with Size, Location, and Neighborhood

# Libraries
import warnings
from glob import glob

import pandas as pd
import seaborn as sns
import wqet_grader
from category_encoders import OneHotEncoder
from IPython.display import VimeoVideo
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted

warnings.simplefilter(action="ignore", category=FutureWarning)
wqet_grader.init("Project 2 Assessment")

# Prepare Data
# create a wrangle function to prepare data
def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)

    # Subset data: Apartments in "Capital Federal", less than 400,000
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)

    # Get place name
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)

    # drop "floor" and "expenses" columns for high null count
    df.drop(columns=["floor", "expenses"], inplace=True)

    # drop high cardinality categorical variables
    df.drop(columns=["operation", "property_type", "currency", "properati_url"], inplace=True)

    # drop leaky features
    df.drop(columns=["price", "price_aprox_local_currency", "price_per_m2", "price_usd_per_m2"], inplace=True)

    # drop colums with multicollinearity
    #df.drop(columns=["surface_total_in_m2", "surface_covered_in_m2"], inplace=True)
    df.drop(columns=["surface_total_in_m2", "rooms"], inplace=True)


    return df

# Assemble files in a list
files = glob("data/buenos-aires-real-estate-*.csv")
files

#Write a list comprehension
frames = [wrangle(file) for file in files]

#Concatenate two or more DataFrames using pandas.
df = pd.concat(frames, ignore_index=True)
print(df.info())
df.head()

# Exploration
df.isnull().sum()/len(df)
df.select_dtypes("object").nunique()
sorted(df.columns)
corr= df.select_dtypes("number").drop(columns="price_aprox_usd").corr()
sns.heatmap(corr)

# Split data
features = ["surface_covered_in_m2","lat", "lon", "neighborhood"]
X_train = df[features]
target = "price_aprox_usd"
y_train = df[target]

# build base line model
y_mean = y_train.mean()
y_pred_baseline = [y_mean]*len(y_train)
print("Mean apt price:", round(y_mean,2))

print("Baseline MAE:", mean_absolute_error(y_train, y_pred_baseline))

# Build an iterative model : Ridge
# instantiate model
model = make_pipeline(
      OneHotEncoder(use_cat_names = True),
      SimpleImputer(),
      Ridge()
)

#fit model to training data
model.fit(X_train,y_train)

#Evaluate model
y_pred_training = model.predict(X_train)
print("Training MAE:", mean_absolute_error(y_train, y_pred_training))

# Test model on test data
X_test = pd.read_csv("data/buenos-aires-test-features.csv")
y_pred_test = pd.Series(model.predict(X_test))
y_pred_test.head()

# Create a function make_prediction that takes four arguments (area, lat, lon, and neighborhood
# ) and returns your model's prediction for an apartment price.

def make_prediction(area, lat, lon, neighborhood):
    data= {
        "surface_covered_in_m2":area,
        "lat": lat,
        "lon": lon,
        "neighborhood":neighborhood
    }
    df = pd.DataFrame(data, index=[0])
    prediction = model.predict(df).round(2)[0]

    return f"Predicted apartment price: ${prediction}"

# Create an interact function in Jupyter Widgets.

interact(
    make_prediction,
    area=IntSlider(
        min=X_train["surface_covered_in_m2"].min(),
        max=X_train["surface_covered_in_m2"].max(),
        value=X_train["surface_covered_in_m2"].mean(),
    ),
    lat=FloatSlider(
        min=X_train["lat"].min(),
        max=X_train["lat"].max(),
        step=0.01,
        value=X_train["lat"].mean(),
    ),
    lon=FloatSlider(
        min=X_train["lon"].min(),
        max=X_train["lon"].max(),
        step=0.01,
        value=X_train["lon"].mean(),
    ),
    neighborhood=Dropdown(options=sorted(X_train["neighborhood"].unique())),
);

# Create an interact function in Jupyter Widgets.

interact(
    make_prediction,
    area=IntSlider(
        min=X_train["surface_covered_in_m2"].min(),
        max=X_train["surface_covered_in_m2"].max(),
        value=X_train["surface_covered_in_m2"].mean(),
        ),
    lat=FloatSlider(
        min=X_train["lat"].min(),
        max=X_train["lat"].max(),
        step=0.01,
        value=X_train["lat"].mean(),
        ),
    lon=FloatSlider(
        min=X_train["lon"].min(),
        max=X_train["lon"].max(),
        step=0.01,
        value=X_train["lon"].mean(),
        ),
    neighborhood=Dropdown(options=sorted(X_train["neighborhood"].unique())),
);